# QUESTIONÁRIO
- Quais as variáveis mais relevantes para o estudo?
- Qual a frequência absoluta/relativa/acumulada das empresas por setor, subsetor e segmento?
- Como estão distribuídas as empresas?
- Qual a frequência absoluta/relativa de papéis por tipo de listagem?

Para uma visualização mais ampla, apresenta-se um gráfico setorial, nele foram condensadas apenas as informações dos setores e segmentos para uma maior granularidade da frequência absoluta relativa a cada variável.

# IMPORTS E CONFIGURAÇÕES

In [5]:
# Import de bibliotecas
from zipfile import ZipFile  # Leitura de arquivos .zipgit 
import requests  # Requisição web
from io import BytesIO  # Leitura de arquivo em bytes
import numpy as np  # Manipulação de dados matriciais
import pandas as pd  # Manipulação de dados
from datetime import date
import plotly.express as px  # Visualização gráfica
import plotly.graph_objects as go  # Visualização gráfica mais avançada



# Configurações das bibliotecas
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None
pd.options.plotting.backend = 'plotly'

# Informações
print(date.today().strftime(format=f'%d/%m/%Y'))

28/02/2024


# EXTRAÇÃO DOS DADOS

In [2]:
# Extrair o arquivo excel da pasta ZIP diretamente do site de download
url = 'https://www.b3.com.br/data/files/57/E6/AA/A1/68C7781064456178AC094EA8/ClassifSetorial.zip'
response = requests.get(url=url)
with ZipFile(BytesIO(response.content)) as fold:
    file = fold.namelist()[0]
    with fold.open(file) as file:
        df = pd.read_excel(io=file, skiprows=6)

# TRANSFORMAÇÃO DOS DADOS

In [3]:
# Renomeando as colunas necessárias
df = df.rename(columns={'SETOR ECONÔMICO': 'SETOR', 'SEGMENTO': 'NOME', 'LISTAGEM': 'CÓDIGO', 'Unnamed: 4': 'LISTAGEM'})[1:-18]
# Preenchendo valores nulos da coluna SEGMENTO com os valores da coluna NOME onde a coluna CÓDIGO FOR vazia
df.loc[(df['CÓDIGO'].isnull()), 'SEGMENTO'] = df.loc[(df['CÓDIGO'].isnull()), 'NOME']
# Removendo linhas com todos as variáveis nulas
df = df.dropna(how='all')
# Preenchendo valores nulos da coluna LISTAGEM com 'Sem classificação'
df.LISTAGEM = df.LISTAGEM.fillna(value='AUSENTE')
# Preenchendo valores nulos com base nas células anteriores
df['SETOR'].ffill(inplace=True)
df['SUBSETOR'].ffill(inplace=True)
df['SEGMENTO'].ffill(inplace=True)
# Removendo valores inconsistentes
df = df.loc[(df.CÓDIGO!='CÓDIGO') & (df.CÓDIGO!='LISTAGEM') & (~df.CÓDIGO.isnull())]
# Formatando todos os dados
df = df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)
# Alterando tipo das variáveis se necessário
df = df.apply(lambda x: x.astype('category') if (x.dtype == 'O') and (x.name not in ['CÓDIGO', 'NOME']) else x)
# Reordenando colunas
df = df[['CÓDIGO', 'NOME', 'SETOR', 'SUBSETOR', 'SEGMENTO', 'LISTAGEM']]
# Reiniciando indexs
df = df.reset_index(drop=True)
# Criando arquivo parquet para acessos posteriores
df.to_parquet(path='./tickers_por_setor.parquet')

# ANÁLISE EXPLORATÓRIA/DESCRITIVA

- Quais as variáveis mais relevantes para o estudo?

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   CÓDIGO    432 non-null    object  
 1   NOME      432 non-null    object  
 2   SETOR     432 non-null    category
 3   SUBSETOR  432 non-null    category
 4   SEGMENTO  432 non-null    category
 5   LISTAGEM  432 non-null    category
dtypes: category(4), object(2)
memory usage: 13.4+ KB


In [5]:
df.memory_usage()

Index        132
CÓDIGO      3456
NOME        3456
SETOR        820
SUBSETOR    1856
SEGMENTO    3200
LISTAGEM     804
dtype: int64

- R: 
Em função a uma análise descritiva dos dados, todas as variáveis são categóricas nominais significativas, porém, as que podem ser sumarizadas, são "SETOR", "SUBSETOR", "SEGMENTO" e "LISTAGEM", já as demais, tratam-se de nomenclaturas individuais.

- Qual a frequência absoluta/relativa/acumulada dos setores?

In [6]:
fa = df.value_counts(subset=['SETOR'], sort=True, ascending=False).rename("count").reset_index()

fig = px.bar(template="plotly_dark",
             title= 'Ranking dos setores com maior quantidade de empresas', 
             data_frame=fa, 
             x='count', 
             y='SETOR', 
             orientation='h',
             color='SETOR',
             text_auto=True)
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='y')
fig.update(layout_showlegend=False)
fig.update_xaxes(title_text='Quantidade')
fig.show()

c:\Users\adria\anaconda3\Lib\site-packages\plotly\express\_core.py:1958: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(required_grouper, sort=False)  # skip one_group groupers


Fa:
Acima consta um gráfico de barras horizontal com a frequência absoluta dos 11 setores da b3, com um total de 432 empresas, com o intuito de demonstrar o ranking dos setores por quantidade de empresas.

In [7]:
fa = df.value_counts(subset=['SETOR'], sort=True, ascending=False).rename("QTD").reset_index()

fig = px.treemap(data_frame=fa,
                 path=[px.Constant('Total'), 'SETOR'],
                 values='QTD',
                 template='plotly_dark')

fig.update_traces(hovertemplate=None, hoverinfo='label+value+percent entry', textinfo='label+value+percent entry')
fig.show()

c:\Users\adria\anaconda3\Lib\site-packages\plotly\express\_core.py:1619: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Fr:
Acima foi disponibilizado um gráfico setorial, dos 11 setores com um total de 432 empresas, com o intuito de demonstrar a participação relativa de cada setor pelo total de empresas.

In [123]:
fac = (df.value_counts(subset='SETOR', normalize=True, sort=True, ascending=True).rename("cumulative").cumsum()*100).round(2)

fig = px.bar(data_frame=fac, 
             title='Frequência relativa acumulada por setor',
             x=fac.index, 
             y=fac.values, 
             color=fac.values, 
             text=fac.values, 
             template='plotly_dark')
fig.update_traces(hovertemplate='%{y:.2f}%', texttemplate='%{text:.1f}%')
fig.update_layout(hovermode='x unified')
fig.update_yaxes(title_text='Frequência acumulada')

fig.show()

Fac:
Já o gráfico acima demonstra a frequência acumulada dos setores por ordem crescente.

Considerações:
Podemos ver através dos gráficos anteriores que há um acumulo aparente entre os 4 maiores setores (consumo cíclico, bens industriais, financeiro e utilidades públicas), comportando 295 de 432 empresas. Em termos relativos, os 4 maiores setores representam cerca de 68% do total das empresas listadas na b3.

- Como estão distribuídas as empresas?

In [28]:
fa = df.value_counts(subset=['SETOR'], sort=True, ascending=False).rename("QTD").reset_index()

fig = px.box(data_frame=fa, template='plotly_dark', x='QTD', points='all', title='Distribuição de frequência')
fig.show()

Distribuição:
Podemos ver, através do gráfico de caixa acima, a distribuição quantitativa setorial da b3, que nos demonstra, também, que 4 dos 11 setores possuem uma maior concentração numérica de empresas se comparados aos demais.

- Quais as medidas centrais dos dados de quantidade de setores?

In [20]:
fa = df.value_counts(subset=['SETOR'], sort=True, ascending=False).rename("QTD").reset_index()
print('Média da quantidade de empresas por setor:', round(fa.QTD.mean()))
print('Mediana da quantidade de empresas por setor:', fa.QTD.median())
print('Moda:', fa.SETOR[0])

Média da quantidade de empresas por setor: 39
Mediana da quantidade de empresas por setor: 31.0
Moda: Consumo Cíclico


- Qual a frequência absoluta/relativa/acumulada dos setores por segmentos?

In [25]:
fa = df.value_counts(subset=['SETOR', 'SEGMENTO'], sort=False).rename("count").reset_index()

fig = px.treemap(data_frame=fa, 
                 path=[px.Constant('Total'), 'SETOR', 'SEGMENTO'], 
                 values='count', 
                 template='plotly_dark')
fig.update_traces(hovertemplate=None, hoverinfo='label+value+percent entry', textinfo='label+value+percent entry')
fig.show()

c:\Users\adria\anaconda3\Lib\site-packages\plotly\express\_core.py:1619: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

c:\Users\adria\anaconda3\Lib\site-packages\plotly\express\_core.py:1619: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



R:
Através do gráfico setorial impresso acima, podemos visualizar com maiores detalhes a relação entre os setores e seus segmentos. Observa-se principalmente os top 4 maiores setores (Consumo cíclico, bens industriais, financeiro e utilidades públicas):
- Consumo cíclico:
- Bens industriais:
- Financeiro:
- Utilidades públicas:
- Consumo não cíclico:
- Materiais básicos:
- Saúde:
- Tecnologia da informação:
- Petróleo, gás e biocombustível:
- Comunicação:
- Outros:

- Qual a frequência absoluta/relativa/acumulada de papéis por tipo de listagem?

In [34]:
fa = df.value_counts(subset=['LISTAGEM'], sort=True, ascending=False).rename("QTD").reset_index()

fig = px.bar(data_frame=fa, 
             template='plotly_dark',
             x='LISTAGEM',
             y='QTD',
             color='LISTAGEM',
             text_auto=True)

fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='x')
fig.show()

c:\Users\adria\anaconda3\Lib\site-packages\plotly\express\_core.py:1958: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [122]:
fac = (df.value_counts(subset='LISTAGEM', normalize=True, sort=True, ascending=True).rename("cumulative").cumsum()*100).round(2)

fig = px.bar(data_frame=fac, x=fac.index, y=fac.values, color=fac.values, text=fac.values, template='plotly_dark')
fig.update_traces(hovertemplate=None, texttemplate='%{text:.1f}%')
fig.show()